## Import

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import pandas as pd
import numpy as np

## Read Data

In [2]:
dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata_large.csv')

In [3]:
dataset.head()

,Age,Salary,Purchased
0,18,20000,0
1,19,22000,0
2,20,24000,0
3,21,28000,0
4,22,60000,1


In [4]:
dataset.describe()

,Age,Salary,Purchased
count,1554.000000,1554.000000,1554.000000
mean,44.296010,57042.471042,0.694981
std,17.462458,21209.244800,0.460564
min,18.000000,20000.000000,0.000000
25%,27.000000,46000.000000,0.000000
50%,43.000000,60000.000000,1.000000
75%,62.000000,66000.000000,1.000000
max,69.000000,96000.000000,1.000000


## Modelling

In [5]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:,-1].values

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=0)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Save scaler for future work

In [8]:
import pickle
pickle.dump(sc, open('sc.pickle', 'wb'))

In [9]:
Xtrain = torch.from_numpy(X_train).float()
Xtest = torch.from_numpy(X_test).float()

In [10]:
ytrain = torch.from_numpy(y_train)
ytest = torch.from_numpy(y_test)

In [11]:
Xtrain.shape, ytrain.shape

(torch.Size([1243, 2]), torch.Size([1243]))

In [12]:
Xtest.shape, ytest.shape

(torch.Size([311, 2]), torch.Size([311]))

In [13]:
input_size=2
output_size=2
hidden_size=10

In [14]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = torch.nn.Linear(input_size, hidden_size)
    self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
    self.fc3 = torch.nn.Linear(hidden_size, output_size)

  def forward(self, X):
    X = torch.relu((self.fc1(X)))
    X = torch.relu((self.fc2(X)))
    X = self.fc3(X)

    return F.log_softmax(X, dim=1)

In [15]:
model = Net()

In [16]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()

In [17]:
epochs = 100

In [18]:
for epoch in range(epochs):
  optimizer.zero_grad()
  Y_pred = model(Xtrain)
  loss = loss_fn(Y_pred, ytrain)
  loss.backward()
  optimizer.step()
  print('Epoch', epoch, 'loss', loss.item())

Epoch 0 loss 0.6319896578788757
Epoch 1 loss 0.6176838278770447
Epoch 2 loss 0.6028789281845093
Epoch 3 loss 0.5874694585800171
Epoch 4 loss 0.5711658596992493
Epoch 5 loss 0.5540099143981934
Epoch 6 loss 0.5365639328956604
Epoch 7 loss 0.5188566446304321
Epoch 8 loss 0.5005489587783813
Epoch 9 loss 0.48166319727897644
Epoch 10 loss 0.4623284339904785
Epoch 11 loss 0.4424836337566376
Epoch 12 loss 0.4219837188720703
Epoch 13 loss 0.40093860030174255
Epoch 14 loss 0.3796481192111969
Epoch 15 loss 0.358404278755188
Epoch 16 loss 0.33749786019325256
Epoch 17 loss 0.31704971194267273
Epoch 18 loss 0.2972123920917511
Epoch 19 loss 0.27842292189598083
Epoch 20 loss 0.26091811060905457
Epoch 21 loss 0.24456259608268738
Epoch 22 loss 0.2293955534696579
Epoch 23 loss 0.21536733210086823
Epoch 24 loss 0.2024935781955719
Epoch 25 loss 0.19075031578540802
Epoch 26 loss 0.1801951378583908
Epoch 27 loss 0.17090246081352234
Epoch 28 loss 0.16284339129924774
Epoch 29 loss 0.15594615042209625
Epoch 30 

In [19]:
list(model.parameters())

[Parameter containing:
 tensor([[ 1.3090,  0.5351],
         [ 0.7109, -0.1853],
         [-0.4728,  0.4106],
         [-0.7684, -0.2693],
         [-1.0414,  0.5446],
         [ 0.2331,  0.3853],
         [-0.4108,  0.6462],
         [-0.2267,  0.2609],
         [-0.7483,  0.2570],
         [-1.2094, -0.9233]], requires_grad=True), Parameter containing:
 tensor([ 0.7579,  0.2359,  0.4254,  0.2693, -0.0072,  1.0517,  0.2857, -0.6245,
          0.0234,  0.2522], requires_grad=True), Parameter containing:
 tensor([[-0.4011,  0.0815, -0.0983,  0.6781, -0.3187,  0.2125, -0.6870,  0.1266,
          -0.4666,  0.6527],
         [-0.4392,  0.2796,  0.1839,  0.5340, -0.2339,  0.0154, -0.3734,  0.1134,
          -0.1649,  0.6617],
         [ 0.6605,  0.7901,  0.2196, -0.2395, -0.0414,  0.4273,  0.4111, -0.0328,
           0.5067, -0.4620],
         [-0.2578,  0.0189, -0.1533,  0.0465,  0.1973, -0.0194, -0.2642,  0.1219,
           0.2307, -0.1462],
         [ 0.2136,  0.5022,  0.3503, -0.4048,  

Check model 

In [20]:
new_data = torch.from_numpy(sc.transform(np.array([[42, 50000]]))).float()

In [21]:
y_cust = model(new_data)
y_cust

tensor([[-0.9611, -0.4820]], grad_fn=<LogSoftmaxBackward>)

In [22]:
_, predicted = torch.max(y_cust, -1)
predicted

tensor([1])

## Export model to ONNX format

In [23]:
sample_tensor = torch.from_numpy(sc.transform(np.array([[40, 20000]]))).float()

In [24]:
torch.onnx.export(model, sample_tensor, "customer.onnx", export_params=True)

In [25]:
!ls

customer.onnx  sample_data  sc.pickle


### Load onnx model to tf

In [26]:
# !pip install onnx

In [40]:
# !pip uninstall onnx_tf

In [26]:
!pip install git+https://github.com/onnx/onnx-tensorflow.git

  Cloning https://github.com/onnx/onnx-tensorflow.git to /tmp/pip-req-build-mozqiql2
  Running command git clone -q https://github.com/onnx/onnx-tensorflow.git /tmp/pip-req-build-mozqiql2
  Created wheel for onnx-tf: filename=onnx_tf-1.7.0-cp37-none-any.whl size=219452 sha256=4c314f092dd2dfb6335cb864a1c07cd2f84ef22326972e1a1a6811cb829cd151
  Stored in directory: /tmp/pip-ephem-wheel-cache-2xwqubu2/wheels/54/24/31/8873b33d2d560efdfa1ed6f346df67ef793b1897358705a480
Successfully built onnx-tf


In [27]:
import onnx
import onnx_tf

In [28]:
# import tensorflow_addons

In [29]:
from onnx_tf.backend import prepare

In [30]:
onnx_model = onnx.load('customer.onnx')

In [31]:
tf_rep = prepare(onnx_model)

In [44]:
prediction = tf_rep.run(np.float32(sc.transform(np.array([[42,50000]]))))

In [45]:
prediction

Outputs(_0=array([[-0.9611399 , -0.48200604]], dtype=float32))

This prediction match the previous pytorch prediction

Save torch model for future work

In [ ]:
torch.save(model, 'customer_buy.pt')

In [ ]:
torch.save(model.state_dict(), 'customer_buy_state_dict')

In [ ]:
# !zip -r customer_buy_state_dict.zip customer_buy_state_dict

In [ ]:
# new_predictor = Net()

In [ ]:
# from google.colab import files


In [ ]:
# files.download('customer_buy_state_dict.zip')